برای سرعت بخشیدن به احرا الگوریتم از کتابخانه تامپای استفاده کردم

In [2]:
import random
import numpy as np

در این الگوریتم در هر مرحله از تولید نسل جدید ۵۰ بار و هردفعه دو بار ۲۰ نفر از اعضا برداشته میشوند و از آن ۲۰ نفر یک فرد لایق تر انتخاب میشود واسه همینم پاپیولیشن همیشه باید بیشتر از ۲۰ باشه.
این تابع به این منظور است که
فشار بر جمعیت کمتر شود چرا که این مسئله دارای نقاط ماکسیمم ناحیه ای است و فقط یک ماکسیمم مطلق ندارد همچنین برای جهش یا همان میوتیشن برای اعداد کمتر از ۶۰ از میوتیشن با شانس دو دهم تا سه دهم استفاده کردم و برای اعداد ۶۰ و صد مقادیر ۰.۱ تا ۰.۷ را تست کردم

برای ۶۰ مقدار ۰.۷ بهترین تنیجه را داد و برای ۱۰۰ نیز از مقدار ۰.۵ استفاده کردم و از آنجا که برای الگوریتم زمان زیادی میبرد با همان مقدار رهایش کردم

همچنین الگوریتم جهش به این صورت عمل میکند که ماکسیمم ۲۰ درصد اعضارا به هم جا به جا میکند (مقدار تکراری در یک کروموزم تولید نمیکند) 

در نتیجه همیشه تضمین میشود که در هر کروموزم ژنوم تکراری وجود نداشته باشه
.
در خصوص تضمین این موضوع در هنگام تولید فرزند جدید نیز چک میکنم ژنوم تکراری در فرزند از والدین نرسیده باشد
برای کراس اور هم دو عدد رندوم انتخاب میکنم و والدین را از آن نقاط میبرم و فرزندان را تولید میکنم


در خصوص پاپیولیشن هم تو اعداد کوچیک اونقدی تاثیر نداشت ولی با ۲۸ واسه ۸ و با ۴۸ برای ۲۰ سریع ترین جواب رو گرفتم

برای ۶۰ اول با ۶۰ تست کردم دیدم همه جامعه شبیه هم شدن و میوتیشن هم خیلی کم و کند اتفاق میوفته 
بعد ۱۲۰ رو تست کردم که پویش بیشتر شه و به نقطه ها ماکسیمم با شانس بیشتری برسیم

فیتنس فانکشن هم اینجوریه که میاد تعداد کانفلیکت های وزیر هارو میشمره و تقسیم بر تعداد کانفلیکت های ممکن میکنه .
اینجوری همیشه یه فیتنس بین صفر و یک دارم اول تو نت دیدم همه اینجوری میزنن بعد تو کد دیدم خیلی بهتره و میتونم یه مقدار ثابت داشته باشم که فهم بهتری داره


حالا تو هرمورد بیشتر توضیح میدم پایین تر

In [70]:

class geneticAlgorithm:
    def __init__(self,pop_size, board_size):
        self.board_size = board_size
        self.board = []
        self.pop_size = pop_size
        self.population=[np.array(random.sample(range(self.board_size),self.board_size)) for i in range(pop_size)]
        self.fitness_arr = [ 0 for i in range(self.pop_size)]
        self.fitness_array_function()

    def sort_pop_by_fitness(self):
        n = self.pop_size
        swapped = False
        for i in range(n):
            for j in range(0,n-i-1):
                if self.fitness_arr[j] > self.fitness_arr[j + 1]:
                    swapped = True
                    self.fitness_arr[j], self.fitness_arr[j + 1] = self.fitness_arr[j + 1], self.fitness_arr[j]
                    self.population[j], self.population[j + 1] = self.population[j + 1], self.population[j]
            if not swapped:
                return

    def fitness_function(self,chromosome):
        number_of_conflicts=0
        for i in range(self.board_size):
            for j in range(i+1,self.board_size):
                if chromosome[i] == chromosome[j]:
                    number_of_conflicts += 1
                if abs(i-j) == abs(chromosome[i]-chromosome[j]):
                    number_of_conflicts += 1
        return 1- ( number_of_conflicts / ( self.board_size*(self.board_size - 1)/2 ) )
        
    def fitness_array_function(self):
        for ind in range(self.pop_size):
            current_fitness = self.fitness_function(self.population[ind])
            self.fitness_arr[ind] = current_fitness
        return self.fitness_arr

    def crossover(self,chromosome_A, chromosome_B):
        n = len(chromosome_A)
        point1 = random.randint(0, n-2)
        point2 = random.randint(point1+1, n-1)
        child1 = [None]*n
        child2 = [None]*n

        child1[point1:point2] = chromosome_A[point1:point2]
        child2[point1:point2] = chromosome_B[point1:point2]


        for i in range(n):
            if child1[i] == None:
                if chromosome_B[i] not in child1:
                    child1[i] = chromosome_B[i]
            if child2[i] == None:
                if chromosome_A[i] not in child2:
                    child2[i] = chromosome_A[i]           


        for i in range(n):
            if child1[i] == None:
                for j in range(n):
                    if j not in child1:
                        child1[i] = j
            if child2[i] == None:
                for j in range(n):
                    if j not in child2:
                        child2[i] = j


        child1_fitness=self.fitness_function(child1)
        child2_fitness=self.fitness_function(child2)
        if child1_fitness>child2_fitness:
            return child1
        return child2


    def mutation(self,chromosome,mutation_prob ):
        
        n = len(chromosome)
        if  random.random()<mutation_prob:
            number_of_mutation = random.randint(0,int(0.2*self.board_size))
            for _ in range (number_of_mutation):
                index1 = random.randint(0,self.board_size-1)
                index2 = random.randint(0,self.board_size-1)
                chromosome[index1],chromosome[index2] = chromosome[index2],chromosome[index1] 
        return chromosome

    def select(self):
        new_generation=[]
        for i in range(0,self.pop_size ):
            random_candidates = np.array(random.sample(range(self.pop_size),20))
            max_index1 = max(random_candidates, key=lambda j: self.fitness_arr[j])
            first_parent = (max_index1,self.population[max_index1],self.fitness_arr[max_index1])
            random_candidates = np.array(random.sample(range(self.pop_size),20))
            max_index2 = max(random_candidates, key=lambda j: self.fitness_arr[j])
            second_parent = (max_index2,self.population[max_index2],self.fitness_arr[max_index2])
            new_child = self.crossover(first_parent[1], second_parent[1])
            new_child = self.mutation(new_child,0.8)
            new_generation.append(new_child)
        new_generation.sort(key=lambda item:self.fitness_function(item))
        self.sort_pop_by_fitness()
        for index in range(len(new_generation)):
            if self.fitness_function(new_generation[index]) > self.fitness_arr[index] : 
                self.population[index] = new_generation[index]
        self.fitness_arr=self.fitness_array_function()
            
    


        
    def print_board (self):
        self.sort_pop_by_fitness()
        arr= self.population[-1]
        for row in arr:
            r = ["[ ]"]*len(arr)
            r[row] = "[Q]"
            print ("".join(r))


واسه هشت خیلی عجیب سریع چواب میده
 احتمالا واسه این باشه که پاپیولیشننم رو از تعداد حالات کانفلیکتش بیشتر دادم 

In [62]:
n = 8
pop = 48
board = geneticAlgorithm(pop,n)
while(max(board.fitness_arr)<1):
    board.select()
board.print_board()

[4, 7, 3, 0, 6, 1, 5, 2]
[ ][ ][ ][ ][Q][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][Q]
[ ][ ][ ][Q][ ][ ][ ][ ]
[Q][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][Q][ ]
[ ][Q][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][Q][ ][ ]
[ ][ ][Q][ ][ ][ ][ ][ ]


اینم خیلی سریع میده با اینکه تعداد پاپیولیشن از تعداد حالات ممکن کانفلیکت کمتره ! 

In [60]:
n = 20
pop = 48
board = geneticAlgorithm(pop,n)
while(max(board.fitness_arr)<1):
    board.select()
board.print_board()

[14, 10, 6, 2, 11, 15, 17, 19, 4, 7, 0, 8, 1, 5, 16, 18, 9, 12, 3, 13]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q]
[ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ]
[ ][ ][ ][ ][ 

از اینجا یکم تایما داره زیاد میشه یه جورایی نماییه دیگه و نمایی بودنشم دیده میشه
تو این حالت با پاپیولیشن ۳۰ اصن جواب نمیگرفتم ولی پاپیولیشن رو که صد گذاشتم خوب شد

In [64]:
n = 40
pop = 100
board = geneticAlgorithm(pop,n)
while(max(board.fitness_arr)<1):
    board.select()
board.print_board()

[35, 9, 20, 5, 7, 34, 15, 13, 28, 19, 22, 38, 26, 30, 1, 39, 14, 10, 23, 32, 4, 2, 12, 6, 24, 36, 18, 31, 29, 17, 37, 0, 8, 27, 25, 21, 11, 16, 33, 3]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ 

واسه شصت یهو خیلی شد !
ولی پاپیولیشن خوب جواب داد و مجبور شدم میوتیشن رو زیاد کنم دفه های اول خیلی دیر به جواب میرسید یه چیزی تو اردر ۴ دقه ولی همینجوری پاپیولیشن رو بیشتر کردم و میوتیشن و بیشتر کردم بهتری شد

میوتیشن ۰.۷ خیلی خوبه براش

In [66]:
n = 60
pop = 120
board = geneticAlgorithm(pop,n)
maxfir=0
while(max(board.fitness_arr)<1):

    board.select()
board.print_board()

[45, 47, 29, 43, 46, 9, 1, 13, 36, 44, 24, 27, 4, 16, 26, 20, 2, 34, 14, 18, 5, 15, 25, 38, 57, 35, 23, 56, 49, 53, 48, 54, 10, 59, 22, 33, 37, 28, 3, 52, 12, 30, 51, 0, 55, 21, 8, 17, 50, 39, 58, 7, 32, 6, 41, 19, 31, 40, 42, 11]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][Q][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]

این خیلی سخت بود 😂
کلی طول کشید تا جواب داد ولی بلاخره داد
میوتیشن هم این سری اخری ۸دهم کردم واسه این بهتر شد جواب
این عجیبه که تو پاپیولیشن اولیمون فیتنس اینقد خوب پیدا میشه!



In [69]:
n = 100
pop = 200
board = geneticAlgorithm(pop,n)
maxfir=0
while(max(board.fitness_arr)<1):
    print(max(board.fitness_arr))
    board.select()
board.print_board()

0.9918325631100707
0.9928426641201716
0.9930446843221918
0.9934487247262322
0.9942568055343131
0.9944588257363333
0.9956709469484545
0.9958729671504747
0.9964790277565353
0.9968830681605757
0.9970850883625959
0.9970850883625959
0.9972871085646161
0.9974891287666363
0.9974891287666363
0.9974891287666363
0.9976911489686565
0.9976911489686565
0.9978931691706767
0.9980951893726969
0.9984992297767373
0.9984992297767373
0.9987012499787575
0.9987012499787575
0.9987012499787575
0.9987012499787575
0.9987012499787575
0.9989032701807777
0.9991052903827979
0.9991052903827979
0.9991052903827979
0.9991052903827979
0.9991052903827979
0.9991052903827979
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9993073105848181
0.9995093307868383
0.9995093307868383
0.9995093307868383
0.9995093307868383
0.9995093307868383
0.9995093307868383
0.9995093307